In [ ]:
import tensorflow.keras as keras
import numpy as np

In [ ]:
keras.backend.set_image_data_format('channels_first')

In [ ]:
def block(last, i):
    x = keras.layers.Conv2D(256, (3, 3), padding="same", name="block{}/conv{}".format(i, 1), activation="relu")(last)
    x = keras.layers.Conv2D(256, (3, 3), padding="same", name="block{}/conv{}".format(i, 2), activation="linear")(x)
    add = keras.layers.Add()([x, last])
    rel = keras.layers.ReLU()(add)
    return rel, add


inputs = keras.Input((112, 8, 8))
last = keras.layers.Conv2D(256, (3, 3), padding="same", name="inputconv")(inputs)
last = keras.layers.ReLU()(last)
inter_outputs = [last]
for b in range(0, 20):
    last, inter = block(last, b)
    inter_outputs.append(inter)

policy_conv = keras.layers.Conv2D(32, (1, 1), activation="relu", padding="same", name="policy_conv")(last)
flatten = keras.layers.Flatten(data_format="channels_last")(policy_conv)
policy_output = keras.layers.Dense(1858, activation="linear", name="policy_dense")(flatten)

In [ ]:
model = keras.Model(inputs, policy_output)

In [ ]:
import onnx

from onnx import numpy_helper
_model = onnx.load("32390.onnx")
INTIALIZERS = _model.graph.initializer
weights = {}
for initializer in INTIALIZERS:
    W = numpy_helper.to_array(initializer)
    if len(W.shape) > 2:
        weights[initializer.name[1:]] = np.moveaxis(W, [0, 1], [-1, -2])
    else:
        weights[initializer.name[1:]] = W

In [ ]:
weights.keys()

In [ ]:
weights["policy/dense/matmul/w"].shape

In [ ]:
weights["output/policy/w"].shape

In [ ]:
np.moveaxis(weights["inputconv/w/kernel"], [0, 1], [-1, -2]).shape

In [ ]:
weights.keys()

In [ ]:
weights["policy/conv/w/kernel"].shape

In [ ]:
for layer in model.layers:
    if "policy_conv" in layer.name:
        layer.set_weights([weights["policy/conv/w/kernel"], weights["policy/conv/w/bias"]])
    elif "policy_dense" in layer.name:
        layer.set_weights([weights["policy/dense/matmul/w"], weights["output/policy/w"]])
    elif "conv" in layer.name:
        layer.set_weights([weights[layer.name + "/w/kernel"], weights[layer.name + "/w/bias"]])

In [ ]:
# Verifying that it gives the same outputs as the onnx model...
import numpy as np
np.random.seed(123)
random_inputs = np.random.random((4, 112, 8, 8)).astype(np.float32)

In [ ]:
tf_outputs = np.array(model(random_inputs))

In [ ]:
import onnxruntime as rt

sess = rt.InferenceSession("32390.onnx", providers=['CUDAExecutionProvider'])
input_name = sess.get_inputs()[0].name
output_names = [[o.name for o in sess.get_outputs()][0]]

In [ ]:
input_name = sess.get_inputs()[0].name

In [ ]:
output_names

In [ ]:
pred = np.squeeze(np.array(sess.run(output_names, {input_name: random_inputs})))

In [ ]:
pred

In [ ]:
tf_outputs

In [ ]:
tf_outputs.shape

In [ ]:
np.allclose(tf_outputs, pred, atol=0.01)

In [ ]:
model.save("policy_model")